In [1]:
import pandas as pd
import numpy as np

inputPath = '../input/'
outputPath = '../output/'

In [2]:
df14Level = pd.read_csv(inputPath + 'studentsAttendance14.csv', index_col=0)
df14Level['stratum2'] = (df14Level['max_transit'] / 60)  > 50
df14Level['stratum3'] = df14Level['num_children'] > 2
df14Level.columns

Index(['USI', 'hmis_id', 'hoh_first_name', 'hoh_last_name', 'created_at',
       'student', 'age', 'school', 'dob', 'first_name_hmis',
       'first_name_hmis_clean', 'last_name_hmis', 'last_name_hmis_clean',
       'user_id', 'site_location', 'num_adults', 'num_children', 'has_car',
       'transportation_car', 'transportation_carpool',
       'transportation_transit', 'transportation_rideshare',
       'transportation_walkbike', 'max_driving', 'max_transit', 'is_treatment',
       'uber', 'num_children_erc', 'num_children_hmis', 'flag', 'School Year',
       'LEA Student ID', 'First Name', 'Middle Name', 'Last Name', 'LEA Code',
       'LEA Name', 'School  Code', 'School Name', 'School Type', 'Grade',
       'DoB', 'Race', 'Gender', 'ELL', 'FARMS', 'At-Risk', 'Attendance Date',
       'Enrollment Date', 'Withdrawal Date', 'Highest SWD Level', 'Ward',
       '_merge', 'daysSinceCreation', 'maxDaysSinceCreation',
       'daysUntilCreation', 'maxDaysUntilCreation', 'present',
       'at

In [3]:
def strata_tables(
    df=None,
    strata_list=[],
    stratum_name='stratum',
    uid='USI',
    treatment = 'is_treatment',
    weights='stratumWeights',
    totalN = 'stratumTotalN',
    treatedN = 'stratumTreatedN',
    prob = 'stratumProb',
    varsToTable=[]):
    
    df = df.copy()
    df.drop_duplicates(subset=uid, inplace=True)
    print(df.shape)
    
    magnitude = 0
    block = pd.Series(np.zeros(shape=df.shape[0]), index=df.index).astype('int')
    for strata_var in strata_list:
        unit=1
        for stratum in df[strata_var].unique():
            block = block + pd.Series(df[strata_var].isin([stratum])) * unit * 10 ** magnitude
            unit += 1
        magnitude += 1
    
    df[stratum_name] = block    
    
    df[totalN] = df[uid].groupby(by=df[stratum_name]).transform('nunique')
    df[treatedN] = df[treatment].groupby(by=df[stratum_name]).transform(np.sum)
    df[prob] = df[treatedN] / df[totalN]
    df[weights] = df[treatment] / df[prob] + (1 - df[treatment]) / (1 - df[prob])
    
    return df

In [5]:
varsToTable = ['stratumTotalN', 'stratumTreatedN', 'stratumProb']
part1 = strata_tables(df=df14Level, uid='USI', strata_list=['stratum2', 'stratum3', 'site_location'])
part1['aggregation'] = 'Student'
part2 = strata_tables(df=df14Level, uid='hmis_id', strata_list=['stratum2', 'stratum3', 'site_location']) 
part2['aggregation'] = 'Family'

together = part1.append(part2)
top = together.pivot_table(values=varsToTable, columns='aggregation', index='stratum', aggfunc='mean', margins=False)
bottom = together.copy()
bottom['stratum'] = 'All'
bottom = bottom.pivot_table(values=varsToTable, columns='aggregation', index='stratum', aggfunc='mean', margins=False)
top.append(bottom)

(382, 61)
(198, 61)


stratumProb           stratumTotalN            stratumTreatedN  \
aggregation      Family   Student        Family    Student          Family   
stratum                                                                      
111            0.500000  0.526316     14.000000  19.000000        7.000000   
112            0.500000  0.428571      6.000000   7.000000        3.000000   
121            0.500000  0.523810      8.000000  21.000000        4.000000   
122            0.625000  0.625000      8.000000  24.000000        5.000000   
211            0.454545  0.500000     11.000000  14.000000        5.000000   
212            0.555556  0.583333      9.000000  12.000000        5.000000   
221            0.461538  0.444444     13.000000  36.000000        6.000000   
222            0.600000  0.551724     10.000000  29.000000        6.000000   
311            0.523810  0.520000     21.000000  25.000000       11.000000   
312            0.454545  0.400000     11.000000  15.000000        5.000000   
321            0.500000  0.562500      4.000000  16.000000        2.000000   
322            0.333333  0.230769      3.000000  13.000000        1.000000   
411            0.500000  0.437500     10.000000  16.000000        5.000000   
412            0.600000  0.625000      5.000000   8.000000        3.000000   
421            0.500000  0.461538      8.000000  26.000000        4.000000   
422            0.555556  0.548387      9.000000  31.000000        5.000000   
511            0.500000  0.478261     18.000000  23.000000        9.000000   
512            0.480000  0.483871     25.000000  31.000000       12.000000   
522            0.400000  0.375000      5.000000  16.000000        2.000000   
All            0.505051  0.494764     13.444444  23.251309        6.752525   

                        
aggregation    Student  
stratum                 
111          10.000000  
112           3.000000  
121          11.000000  
122          15.000000  
211           7.000000  
212           7.000000  
221          16.000000  
222          16.000000  
311          13.000000  
312           6.000000  
321           9.000000  
322           3.000000  
411           7.000000  
412           5.000000  
421          12.000000  
422          17.000000  
511          11.000000  
512          15.000000  
522           6.000000  
All          11.589005

In [16]:
varsToTable = ['age', 'max_driving', 'max_transit'] 
part1 = strata_tables(df=df14Level.loc[df14Level['is_treatment'] == False], uid='USI', strata_list=['stratum2', 'stratum3', 'site_location'])
part1['aggregation'] = 'Control'
part2 = strata_tables(df=df14Level.loc[df14Level['is_treatment'] == True], uid='USI', strata_list=['stratum2', 'stratum3', 'site_location'])
part2['aggregation'] = 'Treatment'

together = part1.append(part2)
together.pivot_table(values=varsToTable, columns='aggregation', index='stratum', aggfunc='mean', margins=True)

(193, 61)
(189, 61)


age                        max_driving               \
aggregation    Control  Treatment        All      Control    Treatment   
stratum                                                                  
111           7.666667   8.200000   7.947368   571.111111   650.300000   
112           7.500000   5.333333   6.571429  1661.250000  1356.666667   
121           7.000000   8.818182   7.952381   432.100000   527.909091   
122           7.444444  10.266667   9.208333  1841.666667  1728.466667   
211           6.714286   7.285714   7.000000   454.571429   290.714286   
212           9.400000   9.285714   9.333333  1048.800000  1376.000000   
221           7.150000   8.750000   7.861111   423.750000   565.500000   
222           8.538462   7.750000   8.103448  1457.230769  1364.437500   
311           7.416667   8.153846   7.800000   584.750000   827.153846   
312           9.666667   8.166667   9.066667  1351.000000  1363.166667   
321           7.285714   7.666667   7.500000  1246.571429   511.333333   
322           6.500000  11.666667   7.692308  1114.500000  1533.000000   
411           7.222222  10.285714   8.562500   664.444444   938.285714   
412          11.333333   9.800000  10.375000  1392.000000  1482.200000   
421           8.642857   8.500000   8.576923   630.785714   655.166667   
422          10.071429  10.647059  10.387097  1273.500000  1352.764706   
511           7.083333   7.909091   7.478261   701.583333   719.909091   
512           7.125000   7.266667   7.193548  1298.062500  1248.066667   
522           8.900000   5.333333   7.562500  1381.100000  1417.500000   
All           7.901554   8.571429   8.232984   970.922280  1021.000000   

                          max_transit                            
aggregation          All      Control    Treatment          All  
stratum                                                          
111           612.789474  1247.222222  1553.700000  1408.526316  
112          1530.714286  4291.250000  3572.333333  3983.142857  
121           482.285714  1198.100000  1403.363636  1305.619048  
122          1770.916667  4204.333333  4011.800000  4084.000000  
211           372.642857  1569.000000   814.142857  1191.571429  
212          1239.666667  3644.600000  4399.142857  4084.750000  
221           486.750000  1135.100000  1588.375000  1336.555556  
222          1406.034483  3756.153846  3818.750000  3790.689655  
311           710.800000  1650.000000  1956.000000  1809.120000  
312          1355.866667  3510.000000  3617.333333  3552.933333  
321           833.000000  2826.428571  2003.666667  2363.625000  
322          1211.076923  3274.500000  4218.000000  3492.230769  
411           784.250000  1832.444444  2405.142857  2083.000000  
412          1448.375000  3313.333333  4012.400000  3750.250000  
421           642.038462  1753.214286  2115.250000  1920.307692  
422          1316.967742  3747.142857  3633.058824  3684.580645  
511           710.347826  2110.000000  1948.000000  2032.521739  
512          1273.870968  3693.125000  3806.133333  3747.806452  
522          1394.750000  4085.500000  3861.000000  4001.312500  
All           995.698953  2650.621762  2795.888889  2722.494764

In [15]:
varsToTable = ['num_adults', 'num_children', 'has_car']
part1 = strata_tables(df=df14Level.loc[df14Level['is_treatment'] == False], uid='hmis_id', strata_list=['stratum2', 'stratum3', 'site_location'])
part1['aggregation'] = 'Control'
part2 = strata_tables(df=df14Level.loc[df14Level['is_treatment'] == True], uid='hmis_id', strata_list=['stratum2', 'stratum3', 'site_location'])
part2['aggregation'] = 'Treatment'

together = part1.append(part2)
together.pivot_table(values=varsToTable, columns='aggregation', index='stratum', aggfunc='mean', margins=True)

(98, 61)
(100, 61)


has_car                     num_adults                      \
aggregation   Control Treatment       All    Control Treatment       All   
stratum                                                                    
111          0.142857  0.142857  0.142857   1.142857  1.142857  1.142857   
112          0.000000  0.000000  0.000000   1.000000  1.333333  1.166667   
121          0.500000  0.250000  0.375000   1.000000  1.250000  1.125000   
122          0.333333  0.400000  0.375000   1.000000  1.400000  1.250000   
211          0.000000  0.600000  0.272727   1.166667  1.400000  1.272727   
212          0.250000  0.600000  0.444444   1.750000  1.400000  1.555556   
221          0.142857  0.500000  0.307692   1.000000  1.666667  1.307692   
222          0.500000  0.333333  0.400000   1.250000  1.333333  1.300000   
311          0.300000  0.454545  0.380952   1.100000  1.181818  1.142857   
312          0.500000  0.000000  0.272727   1.166667  1.200000  1.181818   
321          0.500000  0.000000  0.250000   1.500000  1.000000  1.250000   
322          0.500000  0.000000  0.333333   1.500000  2.000000  1.666667   
411          0.400000  0.400000  0.400000   1.200000  1.600000  1.400000   
412          0.500000  0.333333  0.400000   1.500000  1.333333  1.400000   
421          0.250000  0.250000  0.250000   1.500000  1.500000  1.500000   
422          0.250000  0.600000  0.444444   1.750000  1.200000  1.444444   
511          0.111111  0.000000  0.055556   1.000000  1.444444  1.222222   
512          0.153846  0.000000  0.080000   1.153846  1.083333  1.120000   
522          0.000000  0.500000  0.200000   1.000000  1.000000  1.000000   
All          0.244898  0.280000  0.262626   1.193878  1.310000  1.252525   

            num_children                      
aggregation      Control Treatment       All  
stratum                                       
111             1.285714  1.428571  1.357143  
112             1.333333  1.000000  1.166667  
121             4.500000  3.250000  3.875000  
122             3.333333  3.800000  3.625000  
211             1.500000  1.800000  1.636364  
212             1.500000  1.600000  1.555556  
221             3.428571  3.500000  3.461538  
222             4.000000  4.500000  4.300000  
311             1.200000  1.181818  1.190476  
312             1.500000  1.200000  1.363636  
321             3.500000  5.000000  4.250000  
322             5.500000  3.000000  4.666667  
411             1.800000  1.400000  1.600000  
412             1.500000  1.666667  1.600000  
421             3.750000  3.250000  3.500000  
422             3.750000  3.400000  3.555556  
511             1.333333  1.222222  1.277778  
512             1.230769  1.250000  1.240000  
522             3.333333  3.000000  3.200000  
All             2.193878  2.160000  2.176768

In [111]:
student = df14Level.pivot_table(values='USI', columns='is_treatment', index='site_location', margins=True, aggfunc='count') #.rename(columns={0: 'Control', 1: 'Treatment'})
household = df14Level.drop_duplicates(subset='hmis_id').pivot_table(values='USI', columns='is_treatment', index='site_location', margins=True, aggfunc='count')
together = student.merge(household, left_index=True, right_index=True)
together.columns = pd.MultiIndex.from_product([['Student', 'Household'], ['Control', 'Treatment', 'All']])
together

Student                Household               
              Control Tretament  All   Control Tretament  All
site_location                                                
4                  32        39   71        17        19   36
5                  38        32   70        25        23   48
7                  38        31   69        20        19   39
8                  45        46   91        21        22   43
WJ Rolark          40        41   81        15        17   32
All               193       189  382        98       100  198

In [113]:
df14Diff = pd.read_csv(inputPath + 'studentsAttendance14Diff.csv', index_col=0)
print(df14Diff.shape)
print(df14Diff.drop_duplicates(subset='hmis_id').shape)

(308, 61)
(165, 61)


In [117]:
df30Level = pd.read_csv(inputPath + 'studentsAttendance30.csv', index_col=0)
print(df30Level.shape)
print(df30Level.drop_duplicates(subset='hmis_id').shape)

(357, 59)
(187, 59)


In [118]:
df30Diff = pd.read_csv(inputPath + 'studentsAttendance30Diff.csv', index_col=0)
print(df30Diff.shape)
print(df30Diff.drop_duplicates(subset='hmis_id').shape)

(269, 61)
(148, 61)


In [125]:
print(df14Diff.loc[df14Diff['is_treatment'] == False][['attendanceRate', 'attendanceRateBefore', 'attendanceRateChange']].describe())

       attendanceRate  attendanceRateBefore  attendanceRateChange
count      146.000000            146.000000            146.000000
mean         0.853966              0.845958              0.008007
std          0.222361              0.219060              0.172966
min          0.000000              0.000000             -0.577778
25%          0.800000              0.800000             -0.083036
50%          1.000000              0.900000              0.000000
75%          1.000000              1.000000              0.100000
max          1.000000              1.000000              0.746032


In [127]:
print(df14Diff.loc[df14Diff['is_treatment'] == True][['attendanceRate', 'attendanceRateBefore', 'attendanceRateChange']].describe())

       attendanceRate  attendanceRateBefore  attendanceRateChange
count      162.000000            162.000000            162.000000
mean         0.874346              0.847421              0.026925
std          0.183361              0.220973              0.208226
min          0.111111              0.000000             -0.400000
25%          0.800000              0.777778             -0.108333
50%          1.000000              1.000000              0.000000
75%          1.000000              1.000000              0.125000
max          1.000000              1.000000              0.750000


In [129]:
print(df30Diff.loc[df30Diff['is_treatment'] == False][['attendanceRate', 'attendanceRateBefore', 'attendanceRateChange']].describe())

       attendanceRate  attendanceRateBefore  attendanceRateChange
count      135.000000            135.000000            135.000000
mean         0.850627              0.845743              0.004884
std          0.205782              0.208724              0.171529
min          0.000000              0.000000             -0.500000
25%          0.799499              0.804762             -0.072207
50%          0.928571              0.900000              0.000000
75%          1.000000              1.000000              0.060662
max          1.000000              1.000000              0.803571


In [130]:
print(df30Diff.loc[df30Diff['is_treatment'] == True][['attendanceRate', 'attendanceRateBefore', 'attendanceRateChange']].describe())

       attendanceRate  attendanceRateBefore  attendanceRateChange
count      134.000000            134.000000            134.000000
mean         0.872532              0.861655              0.010876
std          0.176233              0.170393              0.172500
min          0.125000              0.250000             -0.513932
25%          0.808333              0.783333             -0.069196
50%          0.944444              0.933333              0.000000
75%          1.000000              1.000000              0.089015
max          1.000000              1.000000              0.650000
